### 1. Configuração do ambiente e carregamento de dados

In [3]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

In [4]:
# Carregamento da tabela principal
caminho = '../data/raw/dados_brutos.csv'
df_psico = pd.read_csv(caminho)

In [5]:
# Carregamento de dicionários
dic_sid = pd.read_csv('../data/dicionarios/br_bd_diretorios_brasil_cid_10.csv')
dic_municipio = pd.read_csv('../data/dicionarios/br_bd_diretorios_brasil_municipio.csv')
dic_geral = pd.read_csv('../data/dicionarios/br_ms_sia_dicionario.csv')

### 2. Entendimento e limpeza de dados

#### 2.1. Limpeza inicial dos dados

In [6]:
df_psico.columns

Index(['ano', 'mes', 'sigla_uf', 'id_municipio', 'id_estabelecimento_cnes',
       'id_estabelecimento_cnes_familia', 'id_procedimento_ambulatorial',
       'id_servico_especializado', 'id_classificacao_servico',
       'data_inicio_atendimento', 'data_termino_atendimento',
       'permanencia_atendimento', 'motivo_saida_permanencia',
       'data_motivo_saida_permanencia', 'ano_processamento',
       'mes_processamento', 'ano_atendimento', 'mes_atendimento',
       'data_nascimento_paciente', 'id_municipio_residencia_paciente',
       'origem_paciente', 'nacionalidade_paciente', 'tipo_idade',
       'idade_paciente', 'sexo_paciente', 'raca_cor_paciente',
       'etnia_paciente', 'carater_atendimento', 'cid_principal_categoria',
       'cid_principal_subcategoria', 'cid_causas_associadas_categoria',
       'cid_causas_associadas_subcategoria', 'tipo_droga', 'destino_paciente',
       'local_realizacao_atendimento', 'indicador_situacao_rua',
       'indicador_estrategia_familia', 'quant

In [7]:
df_psico.shape

(3717639, 41)

In [8]:
df_psico['ano_atendimento'].unique()

array([2012, 2014, 2015, 2017, 2018, 2019, 2021, 2013, 2016, 2022, 2023,
       2020, 2024, 2025])

In [9]:
# Remoção de anos menores que 2014 e maiores que 2024 (fora da série temporal da análise):
anos_remover = [2012, 2013, 2025]

df_psico = df_psico[~df_psico['ano_atendimento'].isin(anos_remover)]

Verificação e tratamento de nulos

In [10]:
nulos = (df_psico.isnull().sum() / len(df_psico)) * 100 
colunas_nulas = nulos[nulos > 0].sort_values(ascending=False)

print("Percentual de valores nulos por coluna:")
print(colunas_nulas)

Percentual de valores nulos por coluna:
indicador_situacao_rua                100.000000
indicador_estrategia_familia          100.000000
data_motivo_saida_permanencia          98.490448
cid_causas_associadas_categoria        97.649287
cid_causas_associadas_subcategoria     93.884226
data_inicio_atendimento                79.061244
data_termino_atendimento               79.061244
cid_principal_categoria                63.873240
cid_principal_subcategoria             36.126760
dtype: float64


Remoção de colunas mais de 70% nulas e tratamento das colunas referentes ao CID pois fazem parte da análise

In [11]:
df_psico['cid_paciente'] = df_psico['cid_principal_subcategoria'].fillna(df_psico['cid_principal_categoria'])

In [12]:
df_psico['cid_paciente'].isnull().mean().round(2)

np.float64(0.0)

In [13]:
df_psico = df_psico.drop(columns=['cid_principal_categoria', 'cid_principal_subcategoria'])

In [14]:
# Remoção de colunas nulas e que não são necessárias para análise
remover = ['indicador_situacao_rua', 'indicador_estrategia_familia', 'data_motivo_saida_permanencia', 'cid_causas_associadas_categoria', 
           'cid_causas_associadas_subcategoria', 'local_realizacao_atendimento', 'quantidade_atendimentos', 'sigla_uf', 
           'nacionalidade_paciente', 'origem_paciente',  'tipo_idade', 'ano_processamento', 'id_estabelecimento_cnes', 
           'id_estabelecimento_cnes_familia', 'id_classificacao_servico', 'mes_processamento','data_inicio_atendimento', 
           'data_termino_atendimento', 'permanencia_atendimento', 'motivo_saida_permanencia', 'etnia_paciente',
           'carater_atendimento', 'tipo_droga', 'destino_paciente', 'ano', 'mes', 'id_servico_especializado'
           ]
df_psico = df_psico.drop(columns=remover)

In [15]:
df_psico.columns

Index(['id_municipio', 'id_procedimento_ambulatorial', 'ano_atendimento',
       'mes_atendimento', 'data_nascimento_paciente',
       'id_municipio_residencia_paciente', 'idade_paciente', 'sexo_paciente',
       'raca_cor_paciente', 'quantidade_produzida_procedimento',
       'quantidade_aprovada_procedimento', 'quantidade_pacientes',
       'cid_paciente'],
      dtype='object')

In [16]:
df_psico = df_psico[df_psico['quantidade_aprovada_procedimento'] > 0].copy()

In [17]:
df_psico = df_psico[df_psico['quantidade_pacientes'] == 1].copy()

In [18]:
df_psico.head()

,id_municipio,id_procedimento_ambulatorial,ano_atendimento,mes_atendimento,data_nascimento_paciente,id_municipio_residencia_paciente,idade_paciente,sexo_paciente,raca_cor_paciente,quantidade_produzida_procedimento,quantidade_aprovada_procedimento,quantidade_pacientes,cid_paciente
5,4125506,301080208,2014,2,1986-09-09,4125506,27,M,99,1,1,1,F102
6,4113205,301080240,2014,2,1967-01-27,4113205,46,M,3,1,1,1,F102
7,4119509,301080194,2014,2,1980-04-09,4119509,33,F,3,1,1,1,F430
8,4119509,301080208,2014,2,1973-07-14,4119509,39,F,3,1,1,1,F45
9,4119509,301080194,2014,2,1954-10-23,4119509,59,F,3,1,1,1,F32


In [28]:
print(df_psico['idade_paciente'].isnull().sum())
print(df_psico['sexo_paciente'].isnull().sum())
print(df_psico['data_nascimento_paciente'].isnull().sum())

0
0
0


#### 2.2. Junção com dicionários

In [19]:
# Municípios
dic_nomes = dic_municipio.set_index('id_municipio')['nome']

# adiciona nome dos municípios de atendimento para a tabela inicial
df_psico['municipio_atendimento'] = df_psico['id_municipio'].map(dic_nomes)

# adiciona nome dos municípios de residencia dos pacientes para a tabela inicial
df_psico['municipio_residencia'] = df_psico['id_municipio_residencia_paciente'].map(dic_nomes)

In [20]:
df_psico = df_psico.drop(columns=['id_municipio', 'id_municipio_residencia_paciente'])

In [21]:
colunas_chave = ['nome_coluna', 'chave']
dic_geral = dic_geral.drop_duplicates(subset=colunas_chave, keep='first')

In [22]:
raca_cor = dic_geral[dic_geral['nome_coluna'] == 'raca_cor_paciente']
mapa = raca_cor.set_index('chave')['valor']
df_psico['raca_cor_paciente'] = df_psico['raca_cor_paciente'].astype(str).map(mapa)

Remoção de pacientes que não são residentes de Curitiba e Região Metropolitana

In [23]:

df_municipios_validos = ['Curitiba', 'Adrianópolis', 'Agudos do Sul', 'Almirante Tamandaré', 'Araucária', 'Balsa Nova','Bocaiúva do Sul', 
                         'Campina Grande do Sul', 'Campo do Tenente','Campo Largo', 'Campo Magro', 'Cerro Azul', 'Colombo', 'Contenda', 
                         'Doutor Ulysses', 'Fazenda Rio Grande', 'Itaperuçu', 'Lapa', 'Mandirituba', 'Piên', 'Pinhais', 'Piraquara', 'Quatro Barras', 
                         'Rio Branco do Sul', 'Rio Negro', 'São José dos Pinhais', 'Quitandinha', 'Tijucas do Sul', 'Tunas do Paraná']

df_psico = df_psico[df_psico['municipio_residencia'].isin(df_municipios_validos)]

#### 2.2. Análise descritiva dos dados

Análise descritiva dos dados, identificação de inconsistencias, outliners, entre outros.

In [30]:
df_psico.shape

(651745, 13)

In [31]:
df_psico['raca_cor_paciente'].unique()

array(['sem informacao', 'parda', 'branca', 'preta', 'amarela',
       'indigena'], dtype=object)

In [32]:
df_psico['raca_cor_paciente'] = df_psico['raca_cor_paciente'].replace('sem informacao', 'não declarado') 

Análise etária dos pacientes

In [33]:
idades = df_psico['idade_paciente'].sort_values(ascending=True)
print(idades)

557837      0
3588033     0
2027025     0
57060       0
3644747     0
           ..
2978600    96
3655786    96
3663119    96
1957389    96
1258024    97
Name: idade_paciente, Length: 651745, dtype: int64


In [39]:
pacientes = df_psico[df_psico['idade_paciente'] == 1]
columns = ['ano_atendimento', 'data_nascimento_paciente', 'idade_paciente']

pacientes[columns].head()

,ano_atendimento,data_nascimento_paciente,idade_paciente
2253,2014,2014-07-13,1
4782,2017,2015-02-27,1
10665,2021,1986-02-12,1
19213,2022,2019-05-10,1
43905,2015,2015-03-01,1


Há algumas inconsistencias entre a idade do paciente e a data de nascimento

Correção no calculo das idades:

In [40]:
df_psico['data_nascimento_paciente'] = pd.to_datetime(df_psico['data_nascimento_paciente'])

df_datas_atendimento = pd.DataFrame ({ 'year': df_psico['ano_atendimento'], 'month': df_psico['mes_atendimento'], 'day': 1})
df_psico['data_atendimento'] = pd.to_datetime(df_datas_atendimento)

idade = df_psico['data_atendimento'].dt.year - df_psico['data_nascimento_paciente'].dt.year

ajuste = np.where ((df_psico['data_atendimento'].dt.month < df_psico['data_nascimento_paciente'].dt.month) |
    ((df_psico['data_atendimento'].dt.month == df_psico['data_nascimento_paciente'].dt.month) &
     (df_psico['data_atendimento'].dt.day < df_psico['data_nascimento_paciente'].dt.day) ),
      1, 0)

df_psico['idade_paciente'] = idade - ajuste

In [46]:
pd.set_option('display.max_rows', 200)

In [50]:
faixa_etaria = df_psico[df_psico['idade_paciente'] > 80]
faixa_etaria['cid_paciente'].value_counts()

cid_paciente
F99     101
Z000     90
F32      83
F41      46
F102     45
F200     44
F03      41
F29      37
F71      34
F209     33
F329     32
F39      28
F205     27
F51      17
F20      17
F00      17
F31      17
F001     15
F322     13
F320     11
F330     10
F331      7
F411      7
F10       6
F341      6
F321      5
F192      4
F332      4
F312      4
F310      3
F199      3
F068      3
F333      2
G40       2
Z658      2
F603      2
F317      2
F220      2
F023      1
F06       1
G20       1
F44       1
F448      1
F172      1
G90       1
F19       1
F02       1
F72       1
F061      1
F009      1
F191      1
F412      1
F239      1
F190      1
G309      1
F419      1
F349      1
Z659      1
F510      1
F000      1
F059      1
Name: count, dtype: int64